In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from  cv2 import imread
from os import getcwd, listdir, path

from tensorflow.data import Dataset
from tensorflow.keras import optimizers
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator



## Create a data generator

In [ ]:
def get_tf_image(filename,d):
    im = imread(path.join(d,filename))
    return tf.convert_to_tensor(im)

In [ ]:
def generate_from_directory(directory="D:\\customTF\\data\\khletters2",
                            positive_label=None, batch_size=1):
    filenames = listdir(directory)
    for n in range(int(len(filenames)/batch_size)):
        part = filenames[n*batch_size: (n+1)*batch_size]
        labels = [ int(p.split("_")[1]) for p in part]
        if positive_label:
            labels = [L==int(positive_label) for L in labels]
        images = [get_tf_image(f,directory) for f in part]
        yield (images, labels)

In [ ]:
def generate_from_directory_cyclic(directory="D:\\customTF\\data\\khletters2",
                                   positive_label=None, batch_size=1):
    filenames = listdir(directory)
    while True:
        for n in range(int(len(filenames)/batch_size)):
            part = filenames[n*batch_size: (n+1)*batch_size]
            labels = [ int(p.split("_")[1]) for p in part]
            if positive_label:
                labels = [L==int(positive_label) for L in labels]
            images = [get_tf_image(f,directory) for f in part]
            yield (images, labels)

In [ ]:
def get_generator( directory,  rescale = 1/255.,batch_size=20, seed=None):
    img_gen = ImageDataGenerator( rescale =  rescale)
    
    return img_gen.flow_from_directory(
        directory, 
        batch_size=batch_size, 
        #color_mode="rgb",# "grayscale", "rgb", "rgba"
        target_size=(64, 64, ), #pixels in each image
        classes=[str(i) for i in range(1,67,2)],#names of subdirs to look into
        class_mode="categorical", #keep categorical, since many subdirs
        seed=seed #random number 
    )


In [ ]:
train_dir = "data\letters"
test_dir = "data\letters" #should be different data

train_generator = get_generator(train_dir, rescale = 1/255.)
test_generator = get_generator(test_dir, rescale = 1/255.)

In [ ]:
#base_folder= "d:\customTF\data\khletters"
m = tf.data.Dataset.list_files(base_folder)
generator = generate_from_directory("D:\customTF\data\khletters2",
                                   positive_label=None, batch_size=1)
generator_c = generate_from_directory_cyclic("D:\customTF\data\khletters2",
                                   positive_label=None, batch_size=1)

In [ ]:
# Display a few images and labels from the training set

batch = next(train_generator)
batch_images = np.array(batch[0])
batch_labels = np.array(batch[1])
lsun_classes = [str(i) for i in range(1,67,2)]#['classroom', 'conference_room', 'church_outdoor']

plt.figure(figsize=(16,10))
for i in range(len(batch_images)):
    ax = plt.subplot(4, 5, i+1)
    plt.imshow(batch_images[i])
    plt.title(lsun_classes[np.where(batch_labels[i] == 1.)[0][0]])
    plt.axis('off')

In [ ]:
for i in range(1):
    #m =  generator_c.send(None) ## OR:
    m =  next(generator_c)
    im = m[0]
    plt.imshow(im[0])

In [ ]:
def get_model(input_shape, output_shape=(1,),name=None):
    model = Sequential(name=name)
    model.add( Input(input_shape))
    model.add( BatchNormalization(momentum=0.8))
    model.add( Dense(100, activation='relu'))
    model.add( BatchNormalization(momentum=0.8))
    #model.add(  Dense(33, activation='sigmoid'))
    model.add(  Dense(output_shape[0], activation='sigmoid'))
    return model

In [ ]:
m = next(train_generator)
print(m[1][1].shape)


In [ ]:
input_shape = (1, 32, 32, 3)#each letter
input_shape = (32, 32, 3)#each letter
output_shape = (1,)

###Get the right values:
m = next(train_generator)
#print(m[0][0].shape)

input_shape_= m[0][0].shape
output_shape_ = m[1][0].shape

print(input_shape_)
print(output_shape_)

model = get_model(input_shape_, output_shape=output_shape_)
model.summary()

In [ ]:
# Compile the model
optimizer_ = optimizers.Adam(learning_rate=1e-2)
#model.compile(optimizer=optimizer_, loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer_, loss='kl_divergence', metrics=['kullback_leibler_divergence'])


In [ ]:
# Trian the model

train_steps = 10
epochs = 3
num_images = 7128 #300 
batch_size = 1#20

##reset generators:
train_generator = get_generator(train_dir, batch_size=batch_size, rescale = 1/255.)


valid_generator = get_generator(train_dir,batch_size=batch_size, rescale = 1/255.)
test_generator = get_generator(test_dir, batch_size=batch_size, rescale = 1/255.)
spe = num_images // batch_size ##15, #300 images in batches of 20
history = model.fit_generator(train_generator, 
                    validation_data=valid_generator,
                    steps_per_epoch=spe,
                    #callbacks=_callbacks, ##not today :)
                    epochs=epochs)


In [ ]:
model.evaluate(test_generator)

In [ ]:
model.save("model_01_060221")
model.save_weights("model_01_060221_weights")